In [1]:
from util_phys import *
from util_phys_ROOT import *

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
sol = 29.9792458
mpip = 0.13957018
mp = 0.93827203

def etime(evt):
    isc = evt.sc[0]-1
    return evt.sc_t[isc] - evt.sc_r[isc]/sol

def bifp(p):
    return math.sqrt(p**2/(p**2+mp**2))

def dtifp(p, d, t, t0):
    return t - d/(sol*bifp(p)) - t0


In [3]:
def hid_pos_runs(N=None, n0=0):
    n = n0
    hid_branches = [ 'q', 'p', 'tr_time', 'sc', 'sc_sect', 'sc_pd', 'sc_r', 'sc_t' ]
    tname = 'h10clone/h10'
    dir_skims = '~/data/batch/e1f_skim' #'/hdd500/home/data/batch/e1f_skim'
    fn_tmpl = '%s/%%d.root'%dir_skims
    exceptional_runs = [37688]
    runs = [(run, fn_tmpl%run) for run in golden_runs if run not in exceptional_runs]
    nruns = len(runs)
    N = nruns if N is None else N
    while n-n0 < N and n < nruns:
        (run, fn) = runs[n]
        yield (run, root_numpy.root2rec(fn_tmpl%run, tname, hid_branches))
        n += 1
# %time runs = [root_numpy.root2rec(fn_tmpl%grun, tname, bnames) \
#               for grun in golden_runs[0:50] if grun not in exceptional_runs]
# %time hid_pos_runs = [(grun, root_numpy.root2rec(fn_tmpl%grun, tname, hid_branches)) \
#                       for grun in golden_runs if grun not in exceptional_runs]

In [4]:
def positive_particles(run):
    dts = []
    for i, (pS, qS, scS) in enumerate(zip(run.p, run.q, run.sc)):
        for p, q, sc in zip(pS, qS, scS):
            if q > 0 and sc > 0 and p > 0 and p < 5.5:
                #if run.sc_sect[i][sc-1] == 3 and run.sc_pd[i][sc-1] == 24:
                d = run.sc_r[i][sc-1]
                t = run.sc_t[i][sc-1]
                t0 = run.tr_time[i]
                dts.append((run.sc_sect[i][sc-1], run.sc_pd[i][sc-1], dtifp(p, d, t, t0)))
    return dts

In [5]:
%time run_dtsS = [(run, positive_particles(d_run)) for (run, d_run) in hid_pos_runs()]

CPU times: user 50min 30s, sys: 30.2 s, total: 51min 1s
Wall time: 50min 17s


In [16]:
runs, peaks = [], []
for (run, dts) in run_dtsS:
    for sect, pdl, dt in dts:
        if sect == 3 and pdl == 24:
            Y,X = np.histogram(dt, bins=np.arange(-5,5,0.25))
            runs.append(run)
            peak = max(zip(X,Y), key=lambda x: x[1])
            peaks.append(peak[0])
            #print(run, peak)

In [23]:
#plt.plot(runs, peaks, 'k.')


(445126, 445126)

In [30]:
def get_peak_points(h):
    points = []
    pfinder = ROOT.TSpectrum(2, 1)
    npeaks = pfinder.Search(h, 2.5, 'goff', 0.5)
    for i in range(0,npeaks):
        x = pfinder.GetPositionX()[i]
        y = pfinder.GetPositionY()[i]
        points.append((x,y))
    return points

In [3]:
run0, run1 = min(golden_runs), max(golden_runs)
print(run0,run1)

(37658, 38751)


In [ ]:
fout = root_open('pid_pos_per_run.root', 'recreate')
hpeaks = Hist(run1-run0+1, run0-0.5, run1+0.5, name='proton_dt_per_run')

nfiles = 0
nevs = 0

for (ifile, (run, fn)) in enumerate(runs, 1):
    if nfiles > 0 and ifile > nfiles: break

    fout.cd()

    hss = []
    for isect in range(0,6):
        hs = []
        for ipdl in range(0,50):
            h = Hist(40, -10, 10,
                     name='r%d_s%d_pdl%d_pid_pos'%(run, isect+1, ipdl+1))
            hs.append(h)
        hss.append(hs)

    fin = ROOT.TFile(fn)
    es = fin.Get(tname)
    ievt = 0
    while es.GetEvent(ievt):
        ievt += 1
        if nevs > 0 and ievt > nevs: break
        for q, p, sc in zip(bytearray(es.q), es.p, bytearray(es.sc)):
            q, sc = twos_comp(q), twos_comp(sc)
            if q > 0 and sc > 0 and p > 0 and p < 5.5:
                sect = twos_comp(bytearray(es.sc_sect)[sc-1])
                pdl = twos_comp(bytearray(es.sc_pd)[sc-1])
                d = es.sc_r[sc-1]
                t = es.sc_t[sc-1]
                t0 = es.tr_time
                hss[sect-1][pdl-1].Fill(dtifp(p, d, t, t0))
    fin.Close()
    h = hss[2][23]
    peaks = get_peak_points(h)
    dt = max([p[0] for p in peaks]) if len(peaks) > 0 else 0
    hpeaks.Fill(run, dt)

In [ ]:
hpeaks

In [23]:
fout = root_open('pid_pos_per_run.root', 'recreate')

nfiles = 0
nevs = 0

for (ifile, (run, fn)) in enumerate(runs, 1):
    if nfiles > 0 and ifile > nfiles: break

    fout.cd()
    hss = []
    for isect in range(0,6):
        hs = HistStack(name='r%d_s%d_pid_pos'%(run, isect+1))
        for ipdl in range(0,50):
            h = Hist2D(50,0,5,40,-10,10)
            h.name='r%d_s%d_pdl%d_pid_pos'%(run, isect+1, ipdl+1)
            h.set_option('colz')
            hs.Add(h)
        hss.append(hs)

    fin = ROOT.TFile(fn)
    es = fin.Get(tname)
    ievt = 0
    while es.GetEvent(ievt):
        ievt += 1
        if nevs > 0 and ievt > nevs: break
        for q, p, sc in zip(bytearray(es.q), es.p, bytearray(es.sc)):
            q, sc = twos_comp(q), twos_comp(sc)
            if q > 0 and sc > 0 and p > 0 and p < 5.5:
                sect = twos_comp(bytearray(es.sc_sect)[sc-1])
                pdl = twos_comp(bytearray(es.sc_pd)[sc-1])
                d = es.sc_r[sc-1]
                t = es.sc_t[sc-1]
                t0 = es.tr_time
                hss[sect-1][pdl-1].Fill(p, dtifp(p, d, t, t0))
    fin.Close()
    for hs in hss:
        fout.WriteObject(hs, hs.name)

fout.Close()

True

In [19]:
hss[3].Draw('pads colz')

('q', 'p', 'tr_time', 'sc', 'sc_sect', 'sc_pd', 'sc_r', 'sc_t')